In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
df = pd.read_csv('ecommerce-session-bigquery.csv')
df.head()

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,date,visitId,type,productRefundAmount,productQuantity,productPrice,productRevenue,productSKU,v2ProductName,v2ProductCategory,productVariant,currencyCode,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),NaN,NaN,1567.0,82.0,17.0,20170721,1500643403,PAGE,NaN,NaN,0,NaN,9180781,Suitcase Organizer Cubes,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,NaN,NaN,321.0,8.0,NaN,20170413,1492114470,PAGE,NaN,NaN,0,NaN,9180793,26 oz Double Wall Insulated Bottle,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
2,7313828956068851679,Referral,228279,United States,San Francisco,NaN,NaN,927.0,11.0,63.0,20170727,1501198444,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
3,6036794406403793540,Organic Search,1615618,United States,Boulder,NaN,NaN,1616.0,13.0,38.0,20170731,1501522359,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,NaN,NaN,1222.0,45.0,53.0,20170728,1501300311,PAGE,NaN,NaN,0,NaN,9180838,Metal Texture Roller Pen,(not set),(not set),USD,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN


# Most profitable city or province based on the total transaction revenue

In [3]:
grouped = df.groupby('city')['totalTransactionRevenue'].sum()
sorted_grouped = grouped.sort_values(ascending=False)
most_profitable = sorted_grouped.head(1)
most_profitable

city
not available in demo dataset    6.510244e+10
Name: totalTransactionRevenue, dtype: float64

# Detect any anomalies, such as a sharp decrease or increase in the number of transactions for a specific product

In [6]:
product_transactions = df.groupby(['v2ProductName', 'date'])['transactions'].sum().reset_index()
product_stats = product_transactions.groupby('v2ProductName')['transactions'].agg(['mean', 'std']).reset_index()
threshold = 3
product_stats['Anomaly'] = ((product_transactions['transactions'] - product_stats['mean']) / product_stats['std']).abs() > threshold
anomalies = product_stats[product_stats['Anomaly']]
anomalies

,v2ProductName,mean,std,Anomaly
16,Compact Selfie Stick,0.100000,0.316228,True
17,Crunch Noise Dog Toy,0.000000,0.000000,True
27,Google Laptop and Cell Phone Stickers,0.066667,0.258199,True
29,Google Leather Perforated Journal,1.000000,0.000000,True
33,Google Men's 100% Cotton Short Sleeve Hero Tee...,0.000000,0.000000,True
36,Google Men's Performance Full Zip Jacket Black,0.000000,0.000000,True
37,Google Men's Pullover Hoodie Grey,0.000000,0.000000,True
58,Kick Ball,0.000000,0.000000,True


# top products based on the total transaction revenue per day

In [10]:
grouped = df.groupby(['v2ProductName', 'date'])['transactionRevenue'].sum().reset_index()
sorted_grouped = grouped.groupby('date').apply(lambda x: x.sort_values('transactionRevenue', ascending=False))
sorted_grouped

v2ProductName      date  \
date                                                                 
20160801 0                           1 oz Hand Sanitizer  20160801   
         84                           Bottle Opener Clip  20160801   
         93                              Chevron Shopper  20160801   
         198                 Electronics Accessory Pouch  20160801   
         706                                   Kick Ball  20160801   
...                                                  ...       ...   
20170727 783                    Metal Texture Roller Pen  20170727   
20170728 645  Google Women's Short Sleeve Hero Tee Black  20170728   
         784                    Metal Texture Roller Pen  20170728   
20170731 785                    Metal Texture Roller Pen  20170731   
20170801 540                   Google Snapback Hat Black  20170801   

              transactionRevenue  
date                              
20160801 0                   0.0  
         84                  0.0  
         93                  0.0  
         198                 0.0  
         706                 0.0  
...                          ...  
20170727 783                 0.0  
20170728 645                 0.0  
         784                 0.0  
20170731 785                 0.0  
20170801 540                 0.0  

[1173 rows x 3 columns]